## 실습 공통 준비 사항

모든 실습 문제는 seaborn 라이브러리에 내장된 Iris(붓꽃) 데이터셋을 사용합니다.

아래 코드를 실행하여 데이터를 준비하고, numpy 배열로 변환하여 사용하세요.

In [1]:
import seaborn as sns
import numpy as np
import math

# Iris 데이터셋 로드
iris_sns = sns.load_dataset('iris')

# 데이터셋을 특징(X)과 라벨(y)로 분리
features_all = iris_sns.drop('species', axis=1).to_numpy()
labels_all = iris_sns['species'].to_numpy()
feature_names = iris_sns.columns[:-1]

# 데이터 확인
print(f"전체 특징 데이터 shape: {features_all.shape}")
print(f"전체 라벨 데이터 shape: {labels_all.shape}")
print(f"첫 번째 데이터: {features_all[0]}, 라벨: {labels_all[0]}")

전체 특징 데이터 shape: (150, 4)
전체 라벨 데이터 shape: (150,)
첫 번째 데이터: [5.1 3.5 1.4 0.2], 라벨: setosa


## 실습 문제 3.1: 사전 확률 계산하기

**설명**: \*\*사전 확률(Prior Probability)\*\*은 어떤 사건에 대해 새로운 관측이나 증거가 주어지기 전에, 우리가 이미 가지고 있는 확률에 대한 초기 믿음을 의미합니다. 베이즈 정리의 출발점($P(A)$)이 되는 이 값은, 전체 데이터에서 특정 클래스가 차지하는 비율을 계산하여 간단히 구할 수 있습니다.

Iris 데이터셋에서 무작위로 붓꽃 하나를 뽑았을 때, 그 붓꽃이 특정 종(예: 'versicolor')일 확률은 얼마일까요? 전체 `labels_all` 데이터에서 각 품종이 얼마나 차지하는지 계산하여 사전 확률을 구해봅시다.

$$P(\text{종}) = \frac{\text{해당 종의 데이터 수}}{\text{전체 데이터 수}}$$

**요구사항**:

  - `labels_all` 배열을 사용합니다.
  - 특정 품종(`target_class`)이 배열에 몇 번 나타나는지 세어보세요. (힌트: `np.sum(labels == target_class)`)
  - 해당 품종의 개수를 전체 데이터 개수로 나누어 사전 확률을 계산하는 `calculate_prior_probability` 함수를 완성하세요.



In [15]:

def calculate_prior_probability(labels: np.ndarray, target_class: str) -> float:
  """
  전체 데이터에서 특정 클래스의 사전 확률을 계산합니다.

  Args:
    labels: 클래스 라벨이 담긴 numpy 배열 (e.g., labels_all)
    target_class: 확률을 계산할 목표 클래스 이름 (e.g., 'versicolor')

  Returns:
    목표 클래스의 사전 확률
  """
  # 전체 데이터 개수
  d1 = np.sum(labels)
  # 목표 클래스의 개수
  d2 = np.sum(target_class)
  # 사전 확률 계산
  return d1


In [16]:
# 1. 'versicolor' 품종의 사전 확률 계산
prior_versicolor = calculate_prior_probability(labels_all, 'versicolor')
print(f"P('versicolor') = {prior_versicolor:.4f}")

# 2. 'setosa' 품종의 사전 확률 계산
prior_setosa = calculate_prior_probability(labels_all, 'setosa')
print(f"P('setosa') = {prior_setosa:.4f}")

# 3. 'virginica' 품종의 사전 확률 계산
prior_virginica = calculate_prior_probability(labels_all, 'virginica')
print(f"P('virginica') = {prior_virginica:.4f}")


TypeError: the resolved dtypes are not compatible with add.reduce. Resolved (dtype('<U10'), dtype('<U10'), dtype('<U20'))

## 실습 문제 3.2: 가능도(Likelihood) 계산하기

**설명**: \*\*가능도(Likelihood)\*\*는 특정 가설(클래스)이 사실이라고 가정했을 때, 현재와 같은 데이터(증거)가 관측될 확률, 즉 $P(\text{데이터}|\text{가설})$를 의미합니다. 이는 우리의 사전 믿음을 새로운 증거를 통해 얼마나 강화하거나 약화할지 결정하는 중요한 요소입니다.

예를 들어, "어떤 붓꽃이 'setosa' 품종이라는 것을 이미 알고 있을 때(가설), 그 꽃의 꽃잎 길이가 1.6cm 미만일(데이터) 확률은 얼마일까?"를 계산해 봅시다. 이는 'setosa' 품종 중에서 특정 조건을 만족하는 데이터의 비율을 계산하여 구할 수 있습니다.

$$P(\text{증거} | \text{클래스}) = \frac{\text{해당 클래스이면서 증거를 만족하는 데이터 수}}{\text{해당 클래스의 전체 데이터 수}}$$

**요구사항**:

  - `features_all`과 `labels_all` 배열을 사용합니다.
  - "품종이 'setosa'일 때, 'petal length'(2번 인덱스)가 1.6 미만일" 가능도를 계산하세요.
  - 이 로직을 일반화하여, 특정 클래스와 특정 특징의 조건을 받아 가능도를 계산하는 `calculate_likelihood` 함수를 완성하세요. `condition_func`에는 람다 함수를 사용해 보세요.



In [ ]:

def calculate_likelihood(
    features: np.ndarray,
    labels: np.ndarray,
    target_class: str,
    feature_index: int,
    condition_func
) -> float:
  """
  특정 클래스가 주어졌을 때, 특정 조건을 만족하는 데이터의 가능도를 계산합니다.

  Args:
    features: 전체 특징 데이터 배열
    labels: 전체 라벨 데이터 배열
    target_class: 가정이 되는 클래스 (예: 'setosa')
    feature_index: 조건을 검사할 특징의 인덱스
    condition_func: 특징 데이터에 적용할 조건 함수 (lambda)

  Returns:
    P(증거 | 클래스) 형태의 가능도
  """
  # 1. 특정 클래스에 해당하는 데이터만 필터링하기 위한 마스크 생성
  mask = 
  # 2. 해당 클래스의 특징 데이터만 추출
  # 3. 해당 클래스의 전체 데이터 수 계산 (분모)
  # 4. 필터링된 데이터에서 조건을 만족하는 데이터 수 계산 (분자)
  # 특정 feature 열에 조건 함수를 적용

  pass



In [ ]:
# 테스트 1: P(petal_length < 1.6 | species = 'setosa')
# 'petal_length'는 2번 인덱스
likelihood_setosa = calculate_likelihood(
    features_all,
    labels_all,
    'setosa',
    2,
    lambda x: x < 1.6
)
print(f"P(petal_length < 1.6 | 'setosa') = {likelihood_setosa:.4f}")
print("해석: 'setosa' 품종은 꽃잎 길이가 1.6 미만일 확률이 매우 높습니다.")

# 테스트 2: P(petal_length > 4.5 | species = 'versicolor')
likelihood_versicolor = calculate_likelihood(
    features_all,
    labels_all,
    'versicolor',
    2,
    lambda x: x > 4.5
)
print(f"\nP(petal_length > 4.5 | 'versicolor') = {likelihood_versicolor:.4f}")
print("해석: 'versicolor' 품종의 꽃잎 길이는 4.5를 초과하는 경우가 일부 있습니다.")

# 테스트 3: P(petal_length > 4.5 | species = 'virginica')
likelihood_virginica = calculate_likelihood(
    features_all,
    labels_all,
    'virginica',
    2,
    lambda x: x > 4.5
)
print(f"\nP(petal_length > 4.5 | 'virginica') = {likelihood_virginica:.4f}")
print("해석: 'virginica' 품종은 꽃잎 길이가 4.5를 초과할 확률이 매우 높습니다.")

## 실습 문제 3.3: 베이즈 정리의 완성: 사후 확률 계산하기

**설명**: 드디어 3장의 마지막 단계입니다\! \*\*사후 확률(Posterior Probability)\*\*은 사전 확률(초기 믿음)과 가능도(새로운 증거)를 결합하여 얻는 **최종 결론**, 즉 '정보가 갱신된 확률'입니다. 이는 베이즈 정리의 최종 목표이며, $P(\text{가설}|\text{데이터})$로 표현됩니다.

**베이즈 정리**: $P(A|B) = \frac{P(B|A)P(A)}{P(B)}$

이번 실습에서는 "꽃잎 길이(`petal_length`)가 4.5cm를 초과하는 붓꽃을 발견했을 때(증거 B), 이 붓꽃이 'virginica' 품종일(가설 A) 확률은 얼마인가?"를 계산해 봅시다. 복잡한 공식 대신, 데이터의 개수를 직접 세어 만드는 \*\*분할표(Contingency Table)\*\*의 원리를 이용하여 이 사후 확률을 직관적으로 구해봅니다.

$$\text{사후 확률 } P(\text{종} | \text{특징}) = \frac{\text{해당 종이면서 특징을 만족하는 데이터 수}}{\text{해당 특징을 만족하는 전체 데이터 수}}$$

**요구사항**:

  - `features_all`과 `labels_all` 배열을 사용합니다.
  - **증거(B)에 해당하는 전체 데이터 수**: 'petal length'(2번 인덱스)가 4.5cm를 초과하는 모든 붓꽃의 수를 계산하세요.
  - **가설(A)과 증거(B)가 동시에 일어나는 데이터 수**: 'virginica' 품종이면서 'petal length'가 4.5cm를 초과하는 붓꽃의 수를 계산하세요.
  - 위 공식에 따라 사후 확률을 계산하는 `calculate_posterior` 함수를 완성하세요.



In [ ]:
def calculate_posterior(
    features: np.ndarray,
    labels: np.ndarray,
    hypothesis_class: str,
    feature_index: int,
    condition_func
) -> float:
  """
  분할표(counting) 원리를 이용해 사후 확률 P(가설|증거)를 계산합니다.

  Args:
    features: 전체 특징 데이터 배열
    labels: 전체 라벨 데이터 배열
    hypothesis_class: 검증하려는 가설 클래스 (예: 'virginica')
    feature_index: 증거가 되는 특징의 인덱스
    condition_func: 증거 조건을 정의하는 함수 (lambda)

  Returns:
    계산된 사후 확률
  """
  # 1. 사전 확률 P(Class) 계산
  # 2. 가능도 P(Evidence|Class) 계산
  # 3. 증거의 전체 확률 P(Evidence) 계산
  #     P(Evidence) = Σ [P(Evidence|Class_i) * P(Class_i)]
  # 4. 베이즈 정리에 따라 사후 확률 계산

  pass


In [ ]:
# 가설: 품종은 'virginica'이다.
# 증거: 'petal_length' > 4.5 cm 이다.
posterior_virginica = calculate_posterior(
    features_all,
    labels_all,
    'virginica',
    2,
    lambda x: x > 4.5
)

print(f"P(species='virginica' | petal_length > 4.5) = {posterior_virginica:.4f}")


# --- 해석 및 비교 (개선된 출력) ---
# 3.1 함수를 사용하여 사전 확률 계산
prior_virginica = calculate_prior_probability(labels_all, 'virginica')

# 3.2 함수를 사용하여 가능도 계산
likelihood_virginica = calculate_likelihood(
    features_all, labels_all, 'virginica', 2, lambda x: x > 4.5
)

print("\n--- 베이즈 정리 과정 해석 ---")
print(f"1. 사전 확률 (초기 믿음): P(virginica) = {prior_virginica:.4f}")
print("   => 아무 정보가 없을 때, 붓꽃이 'virginica'일 확률은 약 33.3%입니다.\n")

print(f"2. 가능도 (증거의 강력함): P(long_petal|virginica) = {likelihood_virginica:.4f}")
print("   => 'virginica'라는 가정 하에, '긴 꽃잎'이라는 증거가 나타날 확률은 98%로 매우 높습니다. 즉, '긴 꽃잎'은 'virginica'의 강력한 특징입니다.\n")

print(f"3. 사후 확률 (갱신된 믿음): P(virginica|long_petal) = {posterior_virginica:.4f}")
print(f"   => 결론: '긴 꽃잎'이라는 강력한 증거를 반영하여, 이 붓꽃이 'virginica'일 것이라는 우리의 믿음은 {prior_virginica*100:.1f}%에서 {posterior_virginica*100:.1f}%로 크게 상승했습니다.")